In [1]:
import datetime
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')
%matplotlib inline
from dateutil.relativedelta import relativedelta

from finrl.apps import config
from finrl.finrl_meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.finrl_meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.finrl_meta.env_portfolio_allocation.env_portfolio import StockPortfolioEnv
from finrl.drl_agents.stablebaselines3.models import DRLAgent
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline, get_baseline_tdx,convert_daily_return_to_pyfolio_ts

import torch
import plotly.express as px

from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import EfficientFrontier
from pypfopt import expected_returns
from pypfopt import objective_functions
from pyfolio import timeseries

d:\code\python\pyfolio\pyfolio\pos.py:27: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; multipliers will not be applied'


In [2]:
from lutils.stock import LTdxHq

In [3]:
ddf = pd.read_pickle('d:/ddf.pkl').dropna()

In [4]:
ddf

,open,close,high,low,volume,amount,tic,date,day
1103,95.00,82.92,95.00,80.00,552103872.0,4.796717e+10,688981,2020-07-16,3
1104,79.00,77.06,84.90,75.00,219471584.0,1.738815e+10,688981,2020-07-17,4
1105,77.19,79.17,80.51,70.02,228583344.0,1.700523e+10,688981,2020-07-20,0
1106,78.30,78.63,82.89,77.77,161873264.0,1.297766e+10,688981,2020-07-21,1
1107,77.80,79.57,81.78,77.20,133942792.0,1.068249e+10,688981,2020-07-22,2
...,...,...,...,...,...,...,...,...,...
1467,1.15,1.14,1.16,1.12,41564420.0,4.722449e+07,600145,2022-01-12,2
1468,1.14,1.13,1.15,1.12,26707056.0,3.029393e+07,600145,2022-01-13,3
1469,1.08,1.07,1.10,1.07,68835016.0,7.400834e+07,600145,2022-01-14,4
1470,1.02,1.02,1.02,1.02,5638200.0,5.750964e+06,600145,2022-01-17,0


In [5]:
ddf = ddf.sort_values(['date','tic'], ignore_index=True)
ddf.index = ddf.date.factorize()[0]

In [6]:
dl = ddf.loc[-252:,:]
dd = dl.pivot_table(index = 'date',columns = 'tic', values = 'close').pct_change().dropna()
corr = dd.cov().corr()
# corr.style.background_gradient(cmap='coolwarm')

# c1 = corr.abs().unstack().sort_values(ascending = True)
# c1 = corr.unstack().sort_values(ascending = True)

In [7]:
# corr.style.background_gradient(cmap='coolwarm')

In [8]:
# corr.sum().sort_values()

In [9]:
stock_codes = list(corr.sum().sort_values().index.values[:6])

In [10]:
stock_codes

['000921', '600365', '000609', '601868', '002372', '001965']

In [11]:
# stock_codes = ['000921', '002032', '300406', '603789']
# stock_codes = ['000609', '000921', '001965', '002372']

In [12]:
ltdxhq = LTdxHq()

indexs = None
dfs = []
for code in stock_codes:
    df = ltdxhq.get_k_data_daily(code, start='2018-01-01') # 2014-01-01
    
    if indexs is None:
        indexs = df.index
    else:
        indexs = indexs.union(df.index)
    
#     df = df.assign(date = df.index)
#     df = df.assign(day = df.index.weekday)
#     df.date = df.date.dt.strftime('%Y-%m-%d')
    df = df.assign(tic = code)
#     df.index = range(df.shape[0])
    
    dfs.append(df)
    print('----------- over %s min: %s max: %s -----------' % (code, df.index.min(), df.index.max()))

for i, df in enumerate(dfs):
    df = df.reindex(indexs)
    df = df.assign(date = df.index)
    df = df.assign(day = df.index.weekday)
    df.index = range(df.shape[0])
    
    dfs[i] = df.ffill()
    
df = pd.concat(dfs)
# df.index = range(df.shape[0])

ltdxhq.close()

----------- over 000921 min: 2018-01-02 00:00:00 max: 2022-01-19 00:00:00 -----------
----------- over 600365 min: 2018-03-20 00:00:00 max: 2022-01-19 00:00:00 -----------
----------- over 000609 min: 2018-01-02 00:00:00 max: 2022-01-19 00:00:00 -----------
----------- over 601868 min: 2021-09-28 00:00:00 max: 2022-01-19 00:00:00 -----------
----------- over 002372 min: 2018-01-02 00:00:00 max: 2022-01-19 00:00:00 -----------
----------- over 001965 min: 2018-01-02 00:00:00 max: 2022-01-19 00:00:00 -----------


In [13]:
fe = FeatureEngineer(use_technical_indicator=True,
                     use_turbulence=False,
                     user_defined_feature = False)

df = fe.preprocess_data(df)

Successfully added technical indicators


In [14]:
# add covariance matrix as states
df=df.sort_values(['date','tic'], ignore_index=True)
df.index = df.date.factorize()[0]

cov_list = []
return_list = []

# look back is one year
lookback=252
for i in range(lookback, len(df.index.unique())):
    data_lookback = df.loc[i-lookback:i,:]
    price_lookback = data_lookback.pivot_table(index = 'date',columns = 'tic', values = 'close')
    return_lookback = price_lookback.pct_change().dropna()
    return_list.append(return_lookback)

    covs = return_lookback.cov().values
    cov_list.append(covs)


df_cov = pd.DataFrame({'date':df.date.unique()[lookback:],'cov_list': cov_list,'return_list': return_list})
df = df.merge(df_cov, on='date')
df = df.sort_values(['date','tic']).reset_index(drop=True)

In [15]:
df['daily_variance'] = (df.high-df.low) / df.close

In [16]:
dl = df.loc[-90:, :]
dd = dl.pivot_table(index = 'date',columns = 'tic', values = 'close').pct_change().dropna()
dd.cov().corr().style.background_gradient(cmap='coolwarm')

tic,000609,000921,001965,002372
tic,,,,
000609,1.000000,-0.229306,-0.229676,-0.292968
000921,-0.229306,1.000000,-0.396903,0.106719
001965,-0.229676,-0.396903,1.000000,-0.769836
002372,-0.292968,0.106719,-0.769836,1.000000


In [17]:
train = data_split(df, '2019-01-01','2022-01-09') # 2021-07-01 2022-01-01

In [18]:
train.head()

,open,close,high,low,volume,amount,tic,date,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,cov_list,return_list,daily_variance
0,5.27,5.29,5.32,5.25,5759311.0,30399482.0,000609,2019-01-15,1,-0.082550,5.946131,4.840869,48.063900,-59.971539,0.911034,5.441000,5.467333,"[[0.0009595613956845235, 0.0003124109814202052...",tic 000609 000921 001965 00...,0.013233
0,7.34,7.55,7.55,7.22,20004778.0,161089488.0,000921,2019-01-15,1,0.171956,7.579382,5.955618,57.955100,171.516755,60.304291,6.799667,6.725167,"[[0.0009595613956845235, 0.0003124109814202052...",tic 000609 000921 001965 00...,0.043709
0,7.39,7.42,7.43,7.34,1422853.0,11541926.0,001965,2019-01-15,1,-0.008600,7.496838,7.139162,49.760481,27.822945,12.144795,7.363000,7.493500,"[[0.0009595613956845235, 0.0003124109814202052...",tic 000609 000921 001965 00...,0.012129
0,11.70,12.04,12.08,11.68,4112392.0,64759288.0,002372,2019-01-15,1,0.160269,12.085685,10.869315,56.642725,165.038307,13.980941,11.421000,11.001833,"[[0.0009595613956845235, 0.0003124109814202052...",tic 000609 000921 001965 00...,0.033223
1,5.28,5.30,5.59,5.16,13215401.0,70665424.0,000609,2019-01-16,2,-0.076507,5.937906,4.832094,48.186940,-33.431536,13.031968,5.425000,5.483167,"[[0.000959630829728534, 0.00031221297698058804...",tic 000609 000921 001965 00...,0.081132


In [19]:
import numpy as np
import pandas as pd
from gym.utils import seeding
import gym
from gym import spaces
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from stable_baselines3.common.vec_env import DummyVecEnv


class StockPortfolioEnv(gym.Env):
    """A portfolio allocation environment for OpenAI gym

    Attributes
    ----------
        df: DataFrame
            input data
        stock_dim : int
            number of unique stocks
        hmax : int
            maximum number of shares to trade
        initial_amount : int
            start money
        transaction_cost_pct: float
            transaction cost percentage per trade
        reward_scaling: float
            scaling factor for reward, good for training
        state_space: int
            the dimension of input features
        action_space: int
            equals stock dimension
        tech_indicator_list: list
            a list of technical indicator names
        turbulence_threshold: int
            a threshold to control risk aversion
        day: int
            an increment number to control date

    Methods
    -------
    _sell_stock()
        perform sell action based on the sign of the action
    _buy_stock()
        perform buy action based on the sign of the action
    step()
        at each step the agent will return actions, then 
        we will calculate the reward, and return the next observation.
    reset()
        reset the environment
    render()
        use render to return other functions
    save_asset_memory()
        return account value at each time step
    save_action_memory()
        return actions/positions at each time step
        

    """
    metadata = {'render.modes': ['human']}

    def __init__(self,
                 df,
                 stock_dim,
                 hmax,
                 initial_amount,
                 transaction_cost_pct,
                 reward_scaling,
                 state_space,
                 action_space,
                 tech_indicator_list,
                 turbulence_threshold=None,
                 lookback=252,
                 day=0):
        #super(StockEnv, self).__init__()
        #money = 10 , scope = 1
        self.day = day
        self.lookback=lookback
        self.df = df
        self.stock_dim = stock_dim
        self.hmax = hmax
        self.initial_amount = initial_amount
#         self.reward = initial_amount
        self.transaction_cost_pct =transaction_cost_pct
        self.reward_scaling = reward_scaling
        self.state_space = state_space
        self.action_space = action_space
        self.tech_indicator_list = tech_indicator_list

        # action_space normalization and shape is self.stock_dim
        self.action_space = spaces.Box(low=0, high=1,shape=(self.action_space,))
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape = (self.state_space+len(self.tech_indicator_list),self.state_space))

        # load data from a pandas dataframe
        self.data = self.df.loc[self.day,:]
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.terminal = False
        self.turbulence_threshold = turbulence_threshold
        # initalize state: inital portfolio return + individual stock return + individual weights
        self.portfolio_value = self.initial_amount

        # memorize portfolio value each step
        self.asset_memory = [self.initial_amount]
        # memorize portfolio return each step
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1 / self.stock_dim] * self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]]

        
    def step(self, actions):
        self.terminal = self.day >= len(self.df.index.unique()) - 1

        if self.terminal:
            df = pd.DataFrame(self.portfolio_return_memory)
            df.columns = ['daily_return']
#             plt.plot(df.daily_return.cumsum(),'r')
#             plt.savefig('results/cumulative_reward.png')
#             plt.close()
            
#             plt.plot(self.portfolio_return_memory,'r')
#             plt.savefig('results/rewards.png')
#             plt.close()

            print("=================================")
            print("begin_total_asset:{}".format(self.asset_memory[0]))
            print("end_total_asset:{}".format(self.portfolio_value))

            df_daily_return = pd.DataFrame(self.portfolio_return_memory)
            df_daily_return.columns = ['daily_return']
            if df_daily_return['daily_return'].std() != 0:
                sharpe = (252 ** 0.5) * df_daily_return['daily_return'].mean() / df_daily_return['daily_return'].std()
                print("Sharpe: ", sharpe)
            print("=================================")
            
            return self.state, self.reward, self.terminal,{}

        else:
            weights = self.softmax_normalization(actions)
            self.actions_memory.append(weights)
            last_day_memory = self.data

            #load next state
            self.day += 1
            self.data = self.df.loc[self.day,:]
            self.covs = self.data['cov_list'].values[0]
            self.state = np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list], axis=0)
            portfolio_return = sum(((self.data.close.values / last_day_memory.close.values) - 1) * weights)
#             log_portfolio_return = np.log(sum((self.data.close.values / last_day_memory.close.values) * weights))
            # update portfolio value
            new_portfolio_value = self.portfolio_value * (1 + portfolio_return)
#             new_portfolio_value = self.portfolio_value * (1 + log_portfolio_return)
            self.portfolio_value = new_portfolio_value

            # save into memory
            self.portfolio_return_memory.append(portfolio_return)
            self.date_memory.append(self.data.date.unique()[0])
            self.asset_memory.append(new_portfolio_value)

            # the reward is the new portfolio value or end portfolo value
            self.reward = new_portfolio_value / self.initial_amount

        return self.state, self.reward, self.terminal, {}

    def reset(self):
        self.asset_memory = [self.initial_amount]
        self.day = 0
        self.data = self.df.loc[self.day,:]
        # load states
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.portfolio_value = self.initial_amount
        #self.cost = 0
        #self.trades = 0
        self.terminal = False
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1/self.stock_dim]*self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]]
        return self.state
    
    def render(self, mode='human'):
        return self.state
        
    def softmax_normalization(self, actions):
        numerator = np.exp(actions)
        denominator = np.sum(np.exp(actions))
        softmax_output = numerator / denominator
        return softmax_output

    
    def save_asset_memory(self):
        date_list = self.date_memory
        portfolio_return = self.portfolio_return_memory
        #print(len(date_list))
        #print(len(asset_list))
        df_account_value = pd.DataFrame({'date':date_list,'daily_return':portfolio_return})
        return df_account_value

    def save_action_memory(self):
        # date and close price length must match actions length
        date_list = self.date_memory
        df_date = pd.DataFrame(date_list)
        df_date.columns = ['date']
        
        action_list = self.actions_memory
        df_actions = pd.DataFrame(action_list)
        df_actions.columns = self.data.tic.values
        df_actions.index = df_date.date
        #df_actions = pd.DataFrame({'date':date_list,'actions':action_list})
        return df_actions

    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def get_sb_env(self):
        e = DummyVecEnv([lambda: self])
        obs = e.reset()
        return e, obs

In [20]:
stock_dimension = len(train.tic.unique())
state_space = stock_dimension
print(f'Stock Dimension: {stock_dimension}, State Space: {state_space}')

Stock Dimension: 4, State Space: 4


In [21]:
# ['daily_variance', 'change', 'log_volume', 'close','day', 'macd', 'rsi_30', 'boll_ub', 'dx_30']
# ['macd', 'boll_ub', 'boll_lb', 'rsi_30', 'boll_ub', 'dx_30', 'close_30_sma', 'close_60_sma'] # cci_30
tech_indicator_list = ['macd', 'boll_ub', 'boll_lb', 'rsi_30', 'daily_variance', 'dx_30', 'close_30_sma', 'close_60_sma']
env_kwargs = {
    "hmax": 100, 
    "initial_amount": 100000, 
    "transaction_cost_pct": 0, 
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": tech_indicator_list, # config.TECHNICAL_INDICATORS_LIST, 
    "action_space": stock_dimension, 
    "reward_scaling": 1e-1
    
}

e_train_gym = StockPortfolioEnv(df = train, **env_kwargs)

In [22]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


In [23]:
agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.005,
    "learning_rate": 0.0001,
    "batch_size": 128,
}
model_ppo = agent.get_model("ppo", model_kwargs = PPO_PARAMS, tensorboard_log='D:/code/python/logs')

{'n_steps': 2048, 'ent_coef': 0.005, 'learning_rate': 0.0001, 'batch_size': 128}
Using cuda device


In [24]:
agent.train_model(model=model_ppo, tb_log_name='ppo', total_timesteps=100000)

d:\soft\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
d:\soft\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
d:\soft\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
d:\soft\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (

Logging to D:/code/python/logs\ppo_3
begin_total_asset:100000
end_total_asset:172605.73605128503
Sharpe:  0.8245005932336298
begin_total_asset:100000
end_total_asset:178642.94357553954
Sharpe:  0.874437350415242
----------------------------------
| time/              |           |
|    fps             | 423       |
|    iterations      | 1         |
|    time_elapsed    | 4         |
|    total_timesteps | 2048      |
| train/             |           |
|    reward          | 1.3276964 |
----------------------------------
begin_total_asset:100000
end_total_asset:177947.6469179541
Sharpe:  0.8643186004258603
begin_total_asset:100000
end_total_asset:159791.17519601088
Sharpe:  0.7241329911249674
begin_total_asset:100000
end_total_asset:152701.44275830858
Sharpe:  0.6737661345377162
------------------------------------------
| time/                   |              |
|    fps                  | 454          |
|    iterations           | 2            |
|    time_elapsed         | 9         

begin_total_asset:100000
end_total_asset:135817.52200063426
Sharpe:  0.5299925786497413
begin_total_asset:100000
end_total_asset:175233.73019828653
Sharpe:  0.8369708432674229
begin_total_asset:100000
end_total_asset:156875.2862141426
Sharpe:  0.709802241705718
------------------------------------------
| time/                   |              |
|    fps                  | 486          |
|    iterations           | 8            |
|    time_elapsed         | 33           |
|    total_timesteps      | 16384        |
| train/                  |              |
|    approx_kl            | 0.0038684774 |
|    clip_fraction        | 0.0139       |
|    clip_range           | 0.2          |
|    entropy_loss         | -5.71        |
|    explained_variance   | 0.000309     |
|    learning_rate        | 0.0001       |
|    loss                 | 150          |
|    n_updates            | 70           |
|    policy_gradient_loss | -0.00528     |
|    reward               | 1.5115886    |
|    st

begin_total_asset:100000
end_total_asset:179025.76766370804
Sharpe:  0.8659004845882773
begin_total_asset:100000
end_total_asset:176648.53098604127
Sharpe:  0.8454302383999402
begin_total_asset:100000
end_total_asset:152889.9245326216
Sharpe:  0.6726267521150563
------------------------------------------
| time/                   |              |
|    fps                  | 489          |
|    iterations           | 15           |
|    time_elapsed         | 62           |
|    total_timesteps      | 30720        |
| train/                  |              |
|    approx_kl            | 0.0036896248 |
|    clip_fraction        | 0.0121       |
|    clip_range           | 0.2          |
|    entropy_loss         | -5.74        |
|    explained_variance   | 0.000116     |
|    learning_rate        | 0.0001       |
|    loss                 | 168          |
|    n_updates            | 140          |
|    policy_gradient_loss | -0.00521     |
|    reward               | 1.0164101    |
|    s

------------------------------------------
| time/                   |              |
|    fps                  | 490          |
|    iterations           | 21           |
|    time_elapsed         | 87           |
|    total_timesteps      | 43008        |
| train/                  |              |
|    approx_kl            | 0.0035395222 |
|    clip_fraction        | 0.0269       |
|    clip_range           | 0.2          |
|    entropy_loss         | -5.76        |
|    explained_variance   | 3.7e-05      |
|    learning_rate        | 0.0001       |
|    loss                 | 125          |
|    n_updates            | 200          |
|    policy_gradient_loss | -0.00952     |
|    reward               | 1.230881     |
|    std                  | 1.02         |
|    value_loss           | 276          |
------------------------------------------
begin_total_asset:100000
end_total_asset:165002.7729702383
Sharpe:  0.7744097006795541
begin_total_asset:100000
end_total_asset:202851.93552

begin_total_asset:100000
end_total_asset:152200.2530145595
Sharpe:  0.6602555521575045
begin_total_asset:100000
end_total_asset:155438.4558983238
Sharpe:  0.6913324324465765
begin_total_asset:100000
end_total_asset:192309.79664586272
Sharpe:  0.9668869375528953
------------------------------------------
| time/                   |              |
|    fps                  | 490          |
|    iterations           | 28           |
|    time_elapsed         | 117          |
|    total_timesteps      | 57344        |
| train/                  |              |
|    approx_kl            | 0.0059189536 |
|    clip_fraction        | 0.0362       |
|    clip_range           | 0.2          |
|    entropy_loss         | -5.79        |
|    explained_variance   | 4.77e-07     |
|    learning_rate        | 0.0001       |
|    loss                 | 97.9         |
|    n_updates            | 270          |
|    policy_gradient_loss | -0.00649     |
|    reward               | 1.2807298    |
|    st

begin_total_asset:100000
end_total_asset:190953.1042491087
Sharpe:  0.9457515941097714
begin_total_asset:100000
end_total_asset:143921.24063781637
Sharpe:  0.5980277433736647
-----------------------------------------
| time/                   |             |
|    fps                  | 490         |
|    iterations           | 35          |
|    time_elapsed         | 146         |
|    total_timesteps      | 71680       |
| train/                  |             |
|    approx_kl            | 0.004804287 |
|    clip_fraction        | 0.03        |
|    clip_range           | 0.2         |
|    entropy_loss         | -5.81       |
|    explained_variance   | 2.56e-06    |
|    learning_rate        | 0.0001      |
|    loss                 | 93.3        |
|    n_updates            | 340         |
|    policy_gradient_loss | -0.00714    |
|    reward               | 1.6298939   |
|    std                  | 1.04        |
|    value_loss           | 197         |
---------------------------

begin_total_asset:100000
end_total_asset:197409.14801282928
Sharpe:  1.0008812505704425
begin_total_asset:100000
end_total_asset:170073.00876448606
Sharpe:  0.8125870149365434
begin_total_asset:100000
end_total_asset:202807.75500609545
Sharpe:  1.0199220047585236
-----------------------------------------
| time/                   |             |
|    fps                  | 490         |
|    iterations           | 42          |
|    time_elapsed         | 175         |
|    total_timesteps      | 86016       |
| train/                  |             |
|    approx_kl            | 0.004232443 |
|    clip_fraction        | 0.0285      |
|    clip_range           | 0.2         |
|    entropy_loss         | -5.86       |
|    explained_variance   | 1.85e-06    |
|    learning_rate        | 0.0001      |
|    loss                 | 88.3        |
|    n_updates            | 410         |
|    policy_gradient_loss | -0.00676    |
|    reward               | 1.2810707   |
|    std              

------------------------------------------
| time/                   |              |
|    fps                  | 490          |
|    iterations           | 48           |
|    time_elapsed         | 200          |
|    total_timesteps      | 98304        |
| train/                  |              |
|    approx_kl            | 0.0043141064 |
|    clip_fraction        | 0.0311       |
|    clip_range           | 0.2          |
|    entropy_loss         | -5.88        |
|    explained_variance   | 1.25e-06     |
|    learning_rate        | 0.0001       |
|    loss                 | 61.5         |
|    n_updates            | 470          |
|    policy_gradient_loss | -0.00778     |
|    reward               | 1.2288206    |
|    std                  | 1.06         |
|    value_loss           | 137          |
------------------------------------------
begin_total_asset:100000
end_total_asset:162077.94607220238
Sharpe:  0.7418587155364947
begin_total_asset:100000
end_total_asset:159001.0249

In [37]:
trade = data_split(df, '2022-01-09', '2022-01-16') # '2021-12-20', '2023-01-01'
e_trade_gym = StockPortfolioEnv(df = trade, **env_kwargs)

In [38]:
unique_tic = trade.tic.unique()
unique_trade_date = trade.date.unique()

In [39]:
df_daily_return_ppo, df_actions_ppo = DRLAgent.DRL_prediction(model=model_ppo, environment=e_trade_gym)
time_ind = pd.Series(df_daily_return_ppo.date)
ppo_cumpod =(df_daily_return_ppo.daily_return + 1).cumprod() - 1
DRL_strat_ppo = convert_daily_return_to_pyfolio_ts(df_daily_return_ppo)

perf_func = timeseries.perf_stats 


perf_stats_all_ppo = perf_func(returns=DRL_strat_ppo, 
                               factor_returns=DRL_strat_ppo, 
                               positions=None, transactions=None, turnover_denom="AGB")

begin_total_asset:100000
end_total_asset:97871.8077283752
Sharpe:  -20.981903359604587
hit end!
